In [1]:
import os
import ecdsa
import hashlib
import base58

class Wallet:
    def __init__(self):
        """ 새로운 비트코인 월렛 생성 """
        # 1. 개인 키 생성
        self.private_key = self._generate_private_key()
        # 2. 공개 키 생성
        self.public_key = self._generate_public_key(self.private_key)
        # 3. 비트코인 주소 생성
        self.address = self._generate_address(self.public_key)

    def _generate_private_key(self) -> bytes:
        """ 32바이트 랜덤 개인 키 생성 """
        return os.urandom(32)

    def _generate_public_key(self, private_key: bytes) -> bytes:
        """ 타원 곡선 연산을 이용해 공개 키 생성 """
        sk = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)
        vk = sk.verifying_key
        return b'\x04' + vk.to_string()  # 비압축 공개 키

    def _generate_address(self, public_key: bytes) -> str:
        """ 비트코인 주소 생성 (SHA-256 → RIPEMD-160 → Base58Check) """
        # 1. SHA-256 해싱
        sha256_pub = hashlib.sha256(public_key).digest()
        # 2. RIPEMD-160 해싱
        ripemd160_pub = hashlib.new('ripemd160', sha256_pub).digest()
        # 3. 네트워크 바이트 추가 (0x00 for Mainnet)
        network_byte = b'\x00' + ripemd160_pub
        # 4. 더블 SHA-256 해싱하여 체크섬 생성
        checksum = hashlib.sha256(hashlib.sha256(network_byte).digest()).digest()[:4]
        # 5. Base58Check 인코딩
        address = base58.b58encode(network_byte + checksum).decode()
        return address

    def get_private_key_hex(self) -> str:
        """ 개인 키를 16진수 문자열로 반환 """
        return self.private_key.hex()

    def get_public_key_hex(self) -> str:
        """ 공개 키를 16진수 문자열로 반환 """
        return self.public_key.hex()

    def get_address(self) -> str:
        """ 생성된 나만의 코인 주소 반환 """
        return self.address

# 테스트 실행
wallet = Wallet()
print(f"개인 키: {wallet.get_private_key_hex()}")
print(f"공개 키: {wallet.get_public_key_hex()}")
print(f"나만의 코인 지갑 주소: {wallet.get_address()}")


개인 키: 72cf669f9fd8e56b3f4b08e60f2e99a33cabf08ad0fea5b9ad5eee457bf423e7
공개 키: 0467710d507a1b6fab637edf974726e0bb9cd77f1d217c3ea7139b804a65a1ca7860e9ee07f93b0b3e570ae2164e050a6d260c3ab68ad3acf591e19c70adc82fb0
나만의 코인 지갑 주소: 1MKonjvZ34nthmbChNK4eBj5obdSwSBivB


# 	단순 계좌 기반 모델 (Account-based) Transaction

In [ ]:
import json
import ecdsa

class Transaction:
    def __init__(self, sender: str, receiver: str, amount: float):
        """ 트랜잭션 생성 """
        self.sender = sender
        self.receiver = receiver
        self.amount = amount
        self.signature = None  # 서명 추가 예정

    def to_dict(self):
        """ 트랜잭션을 딕셔너리로 변환 """
        return {
            "sender": self.sender,
            "receiver": self.receiver,
            "amount": self.amount
        }

    def sign_transaction(self, private_key_hex: str):
        """ 개인 키로 트랜잭션 서명 """
        private_key_bytes = bytes.fromhex(private_key_hex)
        sk = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)
        message = json.dumps(self.to_dict(), sort_keys=True).encode()
        self.signature = sk.sign(message).hex()

    def verify_transaction(self, public_key_hex: str) -> bool:
        """ 공개 키를 사용하여 서명 검증 """
        public_key_bytes = bytes.fromhex(public_key_hex)

    ## 공개 키가 비압축(65바이트)이면 첫 번째 바이트 제거, 압축(33바이트)이면 그대로 사용
        if len(public_key_bytes) == 65 and public_key_bytes[0] == 0x04:
            public_key_bytes = public_key_bytes[1:]

        try:
            vk = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)
            message = json.dumps(self.to_dict(), sort_keys=True).encode()
            return vk.verify(bytes.fromhex(self.signature), message)
        except ecdsa.BadSignatureError:
            return False
        except Exception as e:
            print(f"❌ 공개 키 오류: {e}")
            return False
        
    def get_sender_private_key(self, sender_wallet_address):
        if sender_wallet_address == self.sender: 
            return self._sender_private_key
        else:
            return None



In [187]:
block_chain = Blockchain() 
block_chain.chain

[{'index': 1,
  'timestamp': '2025-03-11 18:54:28.820244',
  'data': 'genesis block',
  'proof': 1,
  'previous_hash': '0'}]

In [188]:
import json 

for cnt in range(5):
    block_chain.mine_block(
        data = f"{block_chain.chain[-1]['index']}th_block_250311",
        miner_address = wallet.get_address(), 
        miner_public_key = wallet.get_public_key_hex()
    )

In [189]:
block_chain.chain

[{'index': 1,
  'timestamp': '2025-03-11 18:54:28.820244',
  'data': 'genesis block',
  'proof': 1,
  'previous_hash': '0'},
 {'index': 2,
  'timestamp': '2025-03-11 18:54:29.937531',
  'data': '{"name": "1th_block_250311", "transaction": []}',
  'proof': 143123,
  'previous_hash': '53e899c608d127b040ee57cc996fff58c2666af17fb33ea53af1dc81cc74693a'},
 {'index': 3,
  'timestamp': '2025-03-11 18:54:30.031461',
  'data': '{"name": "2th_block_250311", "transaction": [{"sender": "Coinbase", "receiver": "1N7eojVALbe35X8VE5mUXazNM2kp8vBy4M", "amount": 10.0}]}',
  'proof': 42228,
  'previous_hash': '0df67d216cf155088f568f0bd2093dfb01a58b3b6f6505c047611f1e3815df40'},
 {'index': 4,
  'timestamp': '2025-03-11 18:54:30.253460',
  'data': '{"name": "3th_block_250311", "transaction": [{"sender": "Coinbase", "receiver": "1N7eojVALbe35X8VE5mUXazNM2kp8vBy4M", "amount": 10.0}]}',
  'proof': 102936,
  'previous_hash': '8c10ba98c2b71faec60b4b482d678f1b7c1b0eab5410dfdccf94f4e4f3525b46'},
 {'index': 5,
  'ti

In [190]:
receiver_wallet = Wallet()
print(receiver_wallet.get_address())

1F7N3TpCst92qc69yepvVisH8UPfLzTmbp


In [191]:
miner_wallet = Wallet() 
print(miner_wallet.get_address())

15opEeBm2hnwPk1F5rRQ3sGPfJ5kftsYoR


In [ ]:
# 블록체인과 월렛 생성
# blockchain = Blockchain()
sender_wallet = wallet
receiver_wallet = receiver_wallet

print(f"채굴자 주소: {miner_wallet.get_address()}")
print(f"송신자 주소: {sender_wallet.get_address()}")
print(f"수신자 주소: {receiver_wallet.get_address()}")

# 트랜잭션 생성
tx = Transaction(
    sender=sender_wallet.get_address(),
    receiver=receiver_wallet.get_address(),
    amount=2.0
)

# 트랜잭션 서명
tx.sign_transaction(sender_wallet.get_private_key_hex())

# 서명 검증
is_valid = tx.verify_transaction(sender_wallet.get_public_key_hex())
print(is_valid)

# 블록체인에 트랜잭션 추가
is_added = block_chain.add_transaction(tx, sender_wallet.get_public_key_hex())
print(f"트랜잭션 추가 성공 여부: {is_added}")

# 블록 채굴 (마이너 주소는 sender_wallet의 주소 사용)
mined_block = block_chain.mine_block(
    data = f"{block_chain.chain[-1]['index']+1}th_block_250311",
    miner_address = miner_wallet.get_address(), 
    miner_public_key= miner_wallet.get_public_key_hex()
)
print("새로운 블록이 채굴됨:", mined_block)

# 블록체인 출력
print("블록체인 상태:")
for block in block_chain.chain:
    print(block)

채굴자 주소: 15opEeBm2hnwPk1F5rRQ3sGPfJ5kftsYoR
송신자 주소: 1N7eojVALbe35X8VE5mUXazNM2kp8vBy4M
수신자 주소: 1F7N3TpCst92qc69yepvVisH8UPfLzTmbp
True
트랜잭션 추가 성공 여부: True
새로운 블록이 채굴됨: {'index': 9, 'timestamp': '2025-03-11 18:56:46.663397', 'data': '{"name": "9th_block_250311", "transaction": [{"sender": "Coinbase", "receiver": "15opEeBm2hnwPk1F5rRQ3sGPfJ5kftsYoR", "amount": 10.0}, {"sender": "1N7eojVALbe35X8VE5mUXazNM2kp8vBy4M", "receiver": "1F7N3TpCst92qc69yepvVisH8UPfLzTmbp", "amount": 2.0}]}', 'proof': 69726, 'previous_hash': '10a6081d7490c18b7613759513b63002951d2a16a2f20b0a1a853f67c5f9d55b'}
블록체인 상태:
{'index': 1, 'timestamp': '2025-03-11 18:54:28.820244', 'data': 'genesis block', 'proof': 1, 'previous_hash': '0'}
{'index': 2, 'timestamp': '2025-03-11 18:54:29.937531', 'data': '{"name": "1th_block_250311", "transaction": []}', 'proof': 143123, 'previous_hash': '53e899c608d127b040ee57cc996fff58c2666af17fb33ea53af1dc81cc74693a'}
{'index': 3, 'timestamp': '2025-03-11 18:54:30.031461', 'data': '{"name":

In [151]:
block_chain.chain

[{'index': 1,
  'timestamp': '2025-03-11 18:32:15.345509',
  'data': 'genesis block',
  'proof': 1,
  'previous_hash': '0'},
 {'index': 2,
  'timestamp': '2025-03-11 18:32:16.914252',
  'data': '0th_block_250311 []',
  'proof': 95140,
  'previous_hash': '1ddf4e5504ea4924f8c0b50f0a68581c763a339d28fba10cd9c5f96345f47b69'},
 {'index': 3,
  'timestamp': '2025-03-11 18:32:16.927252',
  'data': '1th_block_250311 [{"sender": "Coinbase", "receiver": "1N7eojVALbe35X8VE5mUXazNM2kp8vBy4M", "amount": 10.0}]',
  'proof': 12720,
  'previous_hash': '1e899477584059386e9ceb3faea521787faed394c2d5291521aa8059818d6a2d'},
 {'index': 4,
  'timestamp': '2025-03-11 18:32:16.967253',
  'data': '2th_block_250311 [{"sender": "Coinbase", "receiver": "1N7eojVALbe35X8VE5mUXazNM2kp8vBy4M", "amount": 10.0}]',
  'proof': 38104,
  'previous_hash': '61a8be5aaeb81959386812e9dadef1943a4eaa3f2e98251521bb158688284559'},
 {'index': 5,
  'timestamp': '2025-03-11 18:32:17.067252',
  'data': '3th_block_250311 [{"sender": "Coinb

In [198]:
block_chain.pending_transactions[-1].to_dict()

{'sender': 'Coinbase',
 'receiver': '15opEeBm2hnwPk1F5rRQ3sGPfJ5kftsYoR',
 'amount': 10.0}

In [195]:
check_sender = block_chain.pending_transactions[-1].sender 
check_reciever = block_chain.pending_transactions[-1].receiver
check_amount = block_chain.pending_transactions[-1].amount
check_signature = block_chain.pending_transactions[-1].signature

print(f"{check_sender} -> {check_reciever} : {check_amount} BTC")
print(f"서명: {check_signature}")
print(f"트랜잭션 유효성: {is_valid}")

Coinbase -> 15opEeBm2hnwPk1F5rRQ3sGPfJ5kftsYoR : 10.0 BTC
서명: None
트랜잭션 유효성: True
